In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd

from IPython.display import Markdown

import deploy_portfolio_yaml
from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis import styleguide
from segment_speed_utils.project_vars import SEGMENT_GCS

alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

In [ ]:
# parameters cell
org_name = "City of Santa Monica" 

In [ ]:
SPEEDS_SHAPE = "speeds_by_shape_peak_daytype"
SPEEDS_ROUTE = "speeds_by_route_direction_peak_daytype"

months = ["apr", "oct"]

df = pd.concat(
    [pd.read_parquet(
            f"{SEGMENT_GCS}rollup/{SPEEDS_ROUTE}_{m}2023.parquet",
        filters = [[("organization_name", "==", org_name)]],
        ) for m in months
    ], axis=0, ignore_index=True
).astype({"direction_id": "int"})

In [ ]:
%%capture_parameters
org_name = df.organization_name.iloc[0]

org_name

# {org_name} average speeds by day type and peak
**By `route_id-direction_id-mean_stop_sequence`**

In [ ]:
def base_bar_by_time_bin(df: pd.DataFrame, y_col: str) -> alt.Chart:
    
    this_route = df.route_id.unique()[0]
    this_month = df.month.unique()[0]
    this_direction = df.direction_id.unique()[0]
    selected_colors = cp.CALITP_CATEGORY_BRIGHT_COLORS[2:]
    
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x = alt.X("stop_pair:O", title = "Stop Pair",
                    #axis=alt.Axis(labels=False)
                     #sort = "mean_stop_sequence"
                    ),
             y = alt.Y(f"{y_col}:Q", scale = alt.Scale(domain=[0, 50])),
             color = alt.Color(
                 "weekday_weekend:N", title = "",
                 scale = alt.Scale(range = selected_colors)
             ),
             tooltip = ["organization_name", "route_id", "direction_id", 
                        "stop_pair", y_col]
         ).properties(
             width=350, height=250,
        ).facet(
             alt.Column("peak_offpeak:N"), columns=2,
             title = {
                 "text": f"route_id: {this_route}  direction: {this_direction} speeds",
                 "subtitle": f"month: {this_month}"
             })
        .interactive()
    )
    
    return chart

In [ ]:
def dual_bar_chart(
    df, 
    control_field: str,
    y_col: str
):
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    input_dropdown = alt.binding_select(
        options=df.route_id.unique().tolist(), 
        name='Routes '
    )
        
    # Column that controls the bar charts
    category_selector = alt.selection_multi(
        fields=[control_field], 
        bind=input_dropdown
    )
    
    # Build first chart
    chart1 = base_bar_by_time_bin(
        df[df.direction_id==0].reset_index(drop=True),
        y_col = y_col,
    ).add_params(category_selector).transform_filter(category_selector)
    
    # Build second chart
    chart2 = base_bar_by_time_bin(
        df[df.direction_id==1].reset_index(drop=True),
        y_col = y_col,
    ).transform_filter(category_selector)
    
    chart = alt.vconcat(chart1, chart2)
   
    return chart

## April 2023

In [ ]:
dual_bar_chart(
    df[df.month==4].reset_index(drop=True), 
    "route_id", 
    y_col = "p50_mph"
)

## Oct 2023

In [ ]:
dual_bar_chart(
    df[df.month==10].reset_index(drop=True), 
    "route_id", 
    y_col = "p50_mph"
)